In [37]:
import gmaps
import gmaps.datasets
import numpy as np
import requests
import json
import pandas as pd
import base64

In [38]:
from test_environment_Homeaway import google_api_key

In [53]:
# Importing csv file which has all of the airbnb data from data world.
air = pd.read_csv("Airbnb_Texas_Rentals.csv")

In [40]:
air.head(3)

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...


In [41]:
# Looking for all of the Nan data 
np.where(pd.isnull(air))

(array([   25,    25,    25,   103,   103,   103,   104,   104,   104,
          105,   105,   105,   167,   167,   167,   168,   168,   169,
          169,   169,   170,   170,   170,   171,   171,   171,   172,
          172,   172,   174,   174,   175,   175,   177,   177,   177,
          179,   179,   179,   180,   180,   180,   181,   181,   181,
          342,   342,   342,   343,   343,   343,   344,   344,   344,
          345,   345,   346,   346,   346,   362,   362,   362,   408,
          866,   866,   866,   867,   867,   867,   877,   877,   947,
          947,   947,   954,   954,  1120,  1120,  1120,  1122,  1122,
         1122,  1214,  1214,  1214,  1216,  1216,  1216,  1217,  1217,
         1217,  1218,  1218,  1218,  1219,  1219,  1219,  3849,  6875,
         7501,  9296, 14237, 16811, 17186]),
 array([1, 6, 7, 1, 6, 7, 1, 6, 7, 1, 6, 7, 1, 6, 7, 6, 7, 1, 6, 7, 1, 6,
        7, 1, 6, 7, 1, 6, 7, 6, 7, 6, 7, 1, 6, 7, 1, 6, 7, 1, 6, 7, 1, 6,
        7, 1, 6, 7, 1, 6, 

In [42]:
# Deleting all of the nan data for the columns that are needed for google maps
air.dropna(subset=['latitude'], inplace=True)
air.dropna(subset=['longitude'], inplace=True)

In [43]:
# Removing the $ sign from the average rate column and adding a new column without it
air["average_rate_per_night_float"] = air["average_rate_per_night"].str.replace('$', '')

In [44]:
# Turning just the major city listings into a dataframe
austin = air[air["city"]=="Austin"]
sanantonio = air[air["city"]=="San Antonio"]
houston = air[air["city"]=="Houston"]
dallas = air[air["city"]=="Dallas"]
fortworth = air[air["city"]=="Fort Worth"]
corpus = air[air["city"]=="Corpus Christi"]
nb = air[air["city"]=="New Braunfels"]


In [45]:
austin.head(3)

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,average_rate_per_night_float
16,17,$82,2,Austin,December 2015,"Cute two bedroom with lots of windows, sunny b...",30.309522,-97.731710,Cosy 2 Bedroom in Hyde Park!,https://www.airbnb.com/rooms/18550276?location...,82
28,29,$110,1,Austin,September 2015,Comfortable Traditional Interior Furniture in ...,30.244296,-97.774469,Fabulous Central Austin Condo!,https://www.airbnb.com/rooms/13571814?location...,110
33,34,$119,1,Austin,August 2010,"Just minutes from Austin's main attractions, e...",30.300382,-97.778858,Sunny Central Austin Apartment,https://www.airbnb.com/rooms/46929?location=Co...,119


In [46]:
gmaps.configure(api_key=google_api_key)

In [47]:
# Adding a new average rate column without the evil $ sign
austin["average_rate_per_night_float"] = austin["average_rate_per_night"].str.replace('$', '')

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [48]:
# Giving the gmaps function a float column to work with
weights = austin['average_rate_per_night_float'].astype("float")

In [50]:
locations = austin[['latitude', 'longitude']]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [52]:
weights = houston['average_rate_per_night_float'].astype("float")

locations = houston[['latitude', 'longitude']]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))